In [20]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from openai import OpenAI
import re
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def cargar_contenido_pinecone(archivo_texto):
    # Cargar variables de entorno desde el archivo .env en la raíz del proyecto
    load_dotenv(dotenv_path='../.env')
    
    # Verificar si las claves necesarias están presentes
    if 'OPENAI_API_KEY' not in os.environ or 'PINECONE_API_KEY' not in os.environ:
        raise ValueError("Las claves OPENAI_API_KEY y PINECONE_API_KEY deben estar definidas en el archivo .env")
    
    # Configurar OpenAI
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    
    # Configurar Pinecone
    try:
        pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
        index_name = 'puzzles'
        index = pc.Index(index_name)
        logging.info(f"Conectado al índice existente '{index_name}'.")
    except Exception as e:
        logging.error(f"Error al conectar con Pinecone: {e}")
        return

    # Función para generar embeddings usando OpenAI
    def get_embedding(text):
        try:
            response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
            return response.data[0].embedding
        except Exception as e:
            logging.error(f"Error al generar embedding: {e}")
            return None
    
    # Leer y procesar el archivo de texto
    try:
        with open(archivo_texto, 'r', encoding='utf-8') as file:
            contenido = file.read()
        logging.info(f"Archivo leído correctamente. Longitud del contenido: {len(contenido)} caracteres.")
        logging.info(f"Ejemplo de contenido: {contenido[:100]}...")  # Mostrar los primeros 100 caracteres
    except FileNotFoundError:
        logging.error(f"El archivo {archivo_texto} no se encontró.")
        return
    except Exception as e:
        logging.error(f"Error al leer el archivo: {e}")
        return
    
    # Dividir el contenido en preguntas y respuestas
    pares = re.findall(r'(\d+)\.(.*?)\n\*Respuesta:(.*?)\*', contenido, re.DOTALL)
    if not pares:
        logging.warning("No se encontraron pares de preguntas y respuestas con el patrón actual.")
        # Intenta con un patrón alternativo
        pares = re.findall(r'(\d+)\.(.*?)\nRespuesta:(.*?)(?=\n\d+\.|\Z)', contenido, re.DOTALL)
    
    logging.info(f"Se encontraron {len(pares)} pares de preguntas y respuestas.")
    
    if not pares:
        logging.error("No se pudo extraer ningún par de preguntas y respuestas. Verifica el formato del archivo.")
        return
    
    # Preparar los datos para Pinecone
    vectors_to_upsert = []
    for numero, pregunta, respuesta in pares:
        vector_id = f"pregunta_{numero.strip()}"
        texto_completo = f"Pregunta: {pregunta.strip()} Respuesta: {respuesta.strip()}"
        vector = get_embedding(texto_completo)
        if vector is None:
            logging.warning(f"No se pudo generar embedding para la pregunta {numero}")
            continue
        metadata = {
            "numero": numero.strip(),
            "pregunta": pregunta.strip(),
            "respuesta": respuesta.strip()
        }
        vectors_to_upsert.append((vector_id, vector, metadata))
    
    # Cargar los vectores en Pinecone
    if vectors_to_upsert:
        try:
            index.upsert(vectors=vectors_to_upsert)
            logging.info(f"Se han cargado {len(vectors_to_upsert)} pares de preguntas y respuestas en Pinecone.")
        except Exception as e:
            logging.error(f"Error al cargar vectores en Pinecone: {e}")
            logging.error(f"Número de vectores: {len(vectors_to_upsert)}")
            logging.error(f"Dimensión de los vectores: {len(vectors_to_upsert[0][1]) if vectors_to_upsert else 'N/A'}")
    else:
        logging.warning("No se generaron vectores para cargar. Verifica el contenido del archivo y la generación de embeddings.")


if __name__ == "__main__":
    cargar_contenido_pinecone('puzzles.txt')

    # Ejemplo de búsqueda
    resultados = buscar_en_pinecone("¿Qué es blanco y se puede abrir?")
    for resultado in resultados:
        print(f"Puntuación: {resultado['score']}")
        print(f"Número: {resultado['numero']}")
        print(f"Pregunta: {resultado['pregunta']}")
        print(f"Respuesta: {resultado['respuesta']}")
        print("---")

2024-08-18 19:49:17,043 - INFO - Discovering subpackages in _NamespacePath(['c:\\Users\\Admin\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pinecone_plugins'])
2024-08-18 19:49:17,044 - INFO - Looking for plugins in pinecone_plugins.inference
2024-08-18 19:49:17,045 - INFO - Installing plugin inference into Pinecone
2024-08-18 19:49:17,045 - INFO - Conectado al índice existente 'puzzles'.


2024-08-18 19:49:17,055 - INFO - Archivo leído correctamente. Longitud del contenido: 12036 caracteres.
2024-08-18 19:49:17,056 - INFO - Ejemplo de contenido: 1.Una señorita muy aseñorada, que siempre va en coche y siempre va mojada. ¿Qué cosita es?

Respuest...
2024-08-18 19:49:17,070 - WARNING - No se encontraron pares de preguntas y respuestas con el patrón actual.
2024-08-18 19:49:17,071 - INFO - Se encontraron 110 pares de preguntas y respuestas.
2024-08-18 19:49:17,424 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-18 19:49:17,662 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-18 19:49:17,938 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-18 19:49:18,178 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-18 19:49:18,440 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-1